In [1]:
from numpy import *
from scipy.sparse import *
import numpy as np

In [2]:
URM_all=load_npz("all_playlist_with_tracks_URM.npz")
trackIDs=load("fucking_ordered_tracks.npy")
playlistIDs=load("fucking_ordered_playlist.npy")
trackIDs.size,playlistIDs.size,URM_all

(100000, 45649, <45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 1040522 stored elements in Compressed Sparse Column format>)

In [3]:
#pBuilder=array([]).astype(int32)
#for i,p in enumerate(URM_all):
    #pBuilder=np.hstack((pBuilder,array([i]*p.size)))

In [4]:
#save("pBuilder.npy",pBuilder)
pBuilder=load("pBuilder.npy")
pBuilder.size

1040522

In [5]:
#tBuilder=array([]).astype(int32)
#for p in URM_all:
    #tBuilder=np.hstack((tBuilder,where(p.todense()!=0)[1]))

In [6]:
#save("tBuilder.npy",tBuilder)
tBuilder=load("tBuilder.npy")
tBuilder.size

1040522

In [7]:
rBuilder=np.ones(1040522).astype(int32)
rBuilder

array([1, 1, 1, ..., 1, 1, 1], dtype=int32)

In [8]:
class RandomRecommender(object):

    def fit(self, URM_train):
           
        self.numItems = URM_train.shape[1]
    
    
    def recommend(self, playlist_id, at=5):
        
        #genera 5 numeri da 0 a 99999
        recommended_index = np.random.choice(self.numItems, at)
        #prende le canzoni in quella posizione
        recommended_items = trackIDs[recommended_index]

        return recommended_items

In [9]:
def precision(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

In [10]:
def recall(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

In [11]:
def MAP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

In [12]:
# We pass as paramether the recommender class

def evaluate_algorithm(URM_test, recommender_object, at=5):
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0
    
    num_eval = 0


    for i,playlist_id in enumerate(playlistIDs):
        
        #va alla riga i-esima, che contiene la playlist attuale
        relevant_index = URM_test[i].indices
        #prende le track della playlist i-esima
        relevant_items = trackIDs[relevant_index]
        
        if len(relevant_items)>0:
            
            recommended_items = recommender_object.recommend(playlist_id, at=at)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_MAP += MAP(recommended_items, relevant_items)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval
    
    print("Recommender performance is: Precision = {:.6f}, Recall = {:.6f}, MAP = {:.6f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP)) 


In [13]:
train_test_split = 0.80
numInteractions = URM_all.nnz
train_mask = np.random.choice([True,False], numInteractions, p=[train_test_split, 1-train_test_split])

URM_train=(coo_matrix((rBuilder[train_mask],(pBuilder[train_mask],tBuilder[train_mask]))))
URM_train=URM_train.tocsr()

test_mask=logical_not(train_mask)
URM_test=(coo_matrix((rBuilder[test_mask],(pBuilder[test_mask],tBuilder[test_mask]))))
URM_test=URM_test.tocsr()
URM_train,URM_test

(<45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 832222 stored elements in Compressed Sparse Row format>,
 <45649x100000 sparse matrix of type '<class 'numpy.int32'>'
 	with 208300 stored elements in Compressed Sparse Row format>)

In [14]:
randomRecommender = RandomRecommender()
randomRecommender.fit(URM_train)
evaluate_algorithm(URM_test, randomRecommender)

Recommender performance is: Precision = 0.000062, Recall = 0.000034, MAP = 0.000036
